In [1]:
!pip install numpy pandas tensorflow scikit-learn boruta xgboost lightgbm

In [2]:
# Dependencies installation (run these commands in your terminal)
# pip install pandas numpy scikit-learn tensorflow keras matplotlib seaborn xgboost lightgbm boruta
# pip install imbalanced-learn statsmodels scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os
import warnings
warnings.filterwarnings('ignore')

# Feature Selection Libraries
from sklearn.feature_selection import (
    VarianceThreshold, chi2, f_classif, mutual_info_classif, 
    SelectKBest, RFE, SelectFromModel
)
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from boruta import BorutaPy
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector

# ML and Evaluation
from keras.models import Sequential
from keras.layers import (
    Dense, Dropout, BatchNormalization
)
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

# Load dataset
def load_data(file_path):
    print("Loading dataset...")
    df = pd.read_csv(file_path)
    X = df.drop(columns=['COVID-19'])
    y = df['COVID-19']
    print(f"Dataset loaded: {X.shape[0]} samples, {X.shape[1]} features")
    print(f"Class distribution: {dict(y.value_counts())}")
    return X, y

# Define all feature selection techniques
def get_feature_selectors(X, y, n_features=10):
    print("Initializing feature selection techniques...")
    feature_selectors = {
        "1. Chi-Square": SelectKBest(chi2, k=n_features),
        "2. Mutual Information": SelectKBest(mutual_info_classif, k=n_features),
        "3. Recursive Feature Elimination": RFE(
            estimator=LogisticRegression(solver='liblinear', max_iter=1000, random_state=42),
            n_features_to_select=n_features
        ),
        "4. Lasso": SelectFromModel(
            Lasso(alpha=0.01, random_state=42), max_features=n_features
        ),
        "5. Random Forest Importance": SelectFromModel(
            RandomForestClassifier(n_estimators=100, random_state=42), max_features=n_features
        ),
        "6. Boruta": BorutaPy(
            RandomForestClassifier(n_estimators=100, random_state=42),
            n_estimators='auto', verbose=0, random_state=42
        ),
        "7. Correlation-based": None,  # Custom implementation
        "8. Sequential Forward Selection": SequentialFeatureSelector(
            RandomForestClassifier(n_estimators=50, random_state=42),
            n_features_to_select=n_features,
            direction='forward'
        ),
        "9. XGBoost Importance": SelectFromModel(
            XGBClassifier(n_estimators=100, random_state=42), max_features=n_features
        ),
        "10. LightGBM Importance": SelectFromModel(
            LGBMClassifier(n_estimators=100, random_state=42), max_features=n_features
        )
    }
    return feature_selectors

# Function to select top features using each technique
def select_features(X, y, technique_name, selector, n_features=10):
    print(f"Selecting features using {technique_name}...")
    feature_names = X.columns.tolist()
    
    # Handle special case for Correlation-based selection
    if technique_name == "7. Correlation-based":
        # Calculate correlation of each feature with target
        correlations = []
        for col in X.columns:
            corr = np.abs(X[col].corr(y))
            correlations.append((col, corr))
        
        # Sort by correlation and select top n_features
        correlations.sort(key=lambda x: x[1], reverse=True)
        selected_features = [item[0] for item in correlations[:n_features]]
        feature_importances = [item[1] for item in correlations[:n_features]]
        
    # Handle special case for Boruta
    elif technique_name == "6. Boruta":
        # Boruta requires array input
        X_array = X.values
        selector.fit(X_array, y)
        
        # Get the selected features
        selected_mask = selector.support_
        ranking = selector.ranking_
        
        # Sort by ranking and select top features
        feature_ranking = [(feature, rank) for feature, rank, mask in 
                          zip(feature_names, ranking, selected_mask) if mask]
        feature_ranking.sort(key=lambda x: x[1])
        
        # If Boruta selected fewer than n_features, add more by ranking
        if len(feature_ranking) < n_features:
            additional = [(f, r) for f, r, m in 
                         zip(feature_names, ranking, selected_mask) if not m]
            additional.sort(key=lambda x: x[1])
            feature_ranking.extend(additional[:n_features-len(feature_ranking)])
        
        feature_ranking = feature_ranking[:n_features]
        selected_features = [item[0] for item in feature_ranking]
        feature_importances = [1.0/item[1] for item in feature_ranking]  # Invert ranking for visualization
    
    else:
        # Standard scikit-learn selectors
        try:
            selector.fit(X, y)
            
            # Different selector types have different ways to get selected features
            if hasattr(selector, 'get_support'):
                selected_mask = selector.get_support()
                selected_features = [f for f, selected in zip(feature_names, selected_mask) if selected]
                
                # Get feature importances if available
                if hasattr(selector, 'estimator_') and hasattr(selector.estimator_, 'feature_importances_'):
                    feature_importances = selector.estimator_.feature_importances_[selected_mask]
                elif hasattr(selector, 'scores_'):
                    feature_importances = selector.scores_[selected_mask]
                else:
                    feature_importances = np.ones(len(selected_features))
                    
            elif hasattr(selector, 'coef_'):
                # For models with coefficients like Lasso
                coefs = np.abs(selector.coef_)
                indices = np.argsort(coefs)[::-1][:n_features]
                selected_features = [feature_names[i] for i in indices]
                feature_importances = [coefs[i] for i in indices]
                
            else:
                # Get features from the model itself
                try:
                    importances = getattr(selector, 'feature_importances_', 
                                         getattr(selector, 'coef_', None))
                    if importances is None:
                        importances = np.ones(len(feature_names))
                    
                    # For 2D coefficients (like in multiclass), take the mean
                    if importances.ndim > 1:
                        importances = np.mean(np.abs(importances), axis=0)
                    
                    # Select top features
                    indices = np.argsort(np.abs(importances))[::-1][:n_features]
                    selected_features = [feature_names[i] for i in indices]
                    feature_importances = [np.abs(importances)[i] for i in indices]
                    
                except:
                    # Fallback for other selectors
                    indices = getattr(selector, 'support_', np.arange(min(n_features, len(feature_names))))
                    if len(indices) > n_features:
                        indices = indices[:n_features]
                    selected_features = [feature_names[i] for i in indices]
                    feature_importances = np.ones(len(selected_features))
        
        except Exception as e:
            print(f"Error with {technique_name}: {str(e)}")
            # Default to the first n_features if there's an error
            selected_features = feature_names[:n_features]
            feature_importances = np.ones(n_features)
    
    # Ensure exactly n_features are selected (truncate or pad if necessary)
    if len(selected_features) > n_features:
        selected_features = selected_features[:n_features]
        feature_importances = feature_importances[:n_features]
    elif len(selected_features) < n_features:
        # Add remaining features based on variance
        remaining = [f for f in feature_names if f not in selected_features]
        selected_features.extend(remaining[:n_features-len(selected_features)])
        feature_importances = list(feature_importances) + [0] * (n_features - len(feature_importances))
    
    # Print selected features
    print(f"Top {len(selected_features)} features selected by {technique_name}:")
    for i, (feature, importance) in enumerate(zip(selected_features, feature_importances)):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    return selected_features, feature_importances

# Build the ANN model for a specific set of features
def build_ann_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(16, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train and evaluate model with k-fold cross validation
def train_and_evaluate(X, y, selected_features, technique_name, k=5):
    print(f"\nTraining ANN with features selected by {technique_name}")
    
    # Prepare data for ANN
    X_selected = X[selected_features].values
    y_values = y.values
    
    # K-Fold validation
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'auc': []
    }
    
    # Define early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=0,
        mode='min',
        restore_best_weights=True
    )
    
    # Train and evaluate for each fold
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_selected)):
        print(f"Training fold {fold+1}/{k}...")
        
        X_train, X_test = X_selected[train_idx], X_selected[test_idx]
        y_train, y_test = y_values[train_idx], y_values[test_idx]
        
        # Build and train model
        model = build_ann_model((X_selected.shape[1],))
        model.fit(
            X_train, y_train,
            epochs=50,  # Reduced from 100 for faster execution
            batch_size=32,
            verbose=0,
            validation_split=0.2,
            callbacks=[early_stop]
        )
        
        # Evaluate
        y_pred = model.predict(X_test)
        y_pred_classes = (y_pred > 0.5).astype(int)
        
        # Calculate metrics
        metrics['accuracy'].append(accuracy_score(y_test, y_pred_classes))
        metrics['precision'].append(precision_score(y_test, y_pred_classes))
        metrics['recall'].append(recall_score(y_test, y_pred_classes))
        metrics['f1'].append(f1_score(y_test, y_pred_classes))
        try:
            metrics['auc'].append(roc_auc_score(y_test, y_pred))
        except:
            metrics['auc'].append(0.5)  # Default for failed AUC calculation
    
    # Calculate average metrics
    avg_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    std_metrics = {metric: np.std(values) for metric, values in metrics.items()}
    
    print(f"\nResults for {technique_name}:")
    for metric, value in avg_metrics.items():
        print(f"Average {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")
    
    return avg_metrics

# Plot comparison bar chart
def plot_comparison(all_results):
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    
    # Sort techniques by accuracy
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    # Set up the figure
    plt.figure(figsize=(15, 10))
    
    # Set width of bars
    bar_width = 0.15
    index = np.arange(len(sorted_techniques))
    
    # Colors for different metrics
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    # Plot bars for each metric
    for i, metric in enumerate(metrics_to_plot):
        values = [all_results[technique][metric] for technique in sorted_techniques]
        plt.bar(
            index + i * bar_width, 
            values, 
            bar_width, 
            label=metric.capitalize(),
            color=colors[i]
        )
    
    # Add labels and legend
    plt.xlabel('Feature Selection Technique', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.title('Comparison of Feature Selection Techniques', fontsize=14)
    plt.xticks(
        index + bar_width * 2, 
        [t.split('. ')[1] if '. ' in t else t for t in sorted_techniques],
        rotation=45,
        ha='right'
    )
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)
    plt.ylim(0, 1.0)
    plt.tight_layout()
    
    # Save figure
    plt.savefig('feature_selection_comparison_ANN.png', dpi=300, bbox_inches='tight')
    print("Comparison chart saved as 'feature_selection_comparison_ANN.png'")
    plt.close()

# Plot feature heatmap
def plot_feature_heatmap(all_features, X):
    # Create a matrix of features vs techniques
    techniques = list(all_features.keys())
    all_unique_features = list(set(feature for features in all_features.values() for feature in features))
    
    # Create a matrix with 1 if feature is selected by technique, 0 otherwise
    matrix = np.zeros((len(techniques), len(all_unique_features)))
    
    for i, technique in enumerate(techniques):
        for j, feature in enumerate(all_unique_features):
            if feature in all_features[technique]:
                matrix[i, j] = 1
    
    # Sort features by frequency of selection
    feature_counts = matrix.sum(axis=0)
    sorted_indices = np.argsort(feature_counts)[::-1]
    sorted_features = [all_unique_features[i] for i in sorted_indices]
    sorted_matrix = matrix[:, sorted_indices]
    
    # Create heatmap
    plt.figure(figsize=(20, 12))
    sns.heatmap(
        sorted_matrix,
        cmap='Blues',
        xticklabels=sorted_features,
        yticklabels=[t.split('. ')[1] if '. ' in t else t for t in techniques],
        cbar_kws={'label': 'Selected'}
    )
    plt.title('Feature Selection by Different Techniques', fontsize=14)
    plt.tight_layout()
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    
    # Save figure
    plt.savefig('feature_selection_heatmap_ANN.png', dpi=300, bbox_inches='tight')
    print("Heatmap saved as 'feature_selection_heatmap_ANN.png'")
    plt.close()

# Main function to run the whole process
def main():
    # Load data
    file_path = "OneDrive/Desktop/Research_Capstone_Project/preprocessed_covid500_final.csv"  # Update with your path
    X, y = load_data(file_path)
    
    # Get feature selectors
    feature_selectors = get_feature_selectors(X, y)
    
    # Store results
    all_results = {}
    all_selected_features = {}
    
    # For each technique, select features and train model
    for technique_name, selector in feature_selectors.items():
        print("\n" + "="*50)
        print(f"Processing {technique_name}")
        print("="*50)
        
        # Select features
        selected_features, _ = select_features(X, y, technique_name, selector)
        all_selected_features[technique_name] = selected_features
        
        # Train and evaluate
        results = train_and_evaluate(X, y, selected_features, technique_name)
        all_results[technique_name] = results
    
    # Plot comparison
    plot_comparison(all_results)
    
    # Plot feature heatmap
    plot_feature_heatmap(all_selected_features, X)
    
    # Print final summary
    print("\n" + "="*50)
    print("FINAL SUMMARY")
    print("="*50)
    
    # Sort techniques by accuracy
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    print("\nTechniques ranked by accuracy:")
    for i, technique in enumerate(sorted_techniques):
        print(f"{i+1}. {technique}: {all_results[technique]['accuracy']:.4f}")
    
    best_technique = sorted_techniques[0]
    print(f"\nBest performing technique: {best_technique}")
    print(f"Top 10 features selected by {best_technique}:")
    for i, feature in enumerate(all_selected_features[best_technique]):
        print(f"{i+1}. {feature}")

if __name__ == "__main__":
    main()

Loading dataset...
Dataset loaded: 37028 samples, 20 features
Class distribution: {1: 18514, 0: 18514}
Initializing feature selection techniques...

Processing 1. Chi-Square
Selecting features using 1. Chi-Square...
Top 10 features selected by 1. Chi-Square:
1. Breathing Problem: 10855.6785
2. Sore throat: 13892.5526
3. Chronic Lung Disease: 3.2782
4. Heart Disease: 4165.5609
5. Hyper Tension: 3684.8322
6. Attended Large Gathering: 9772.7294
7. Visited Public Exposed Places: nan
8. Family working in Public Exposed Places: 8644.1025
9. Wearing Masks: nan
10. Sanitization from Market: nan

Training ANN with features selected by 1. Chi-Square
Training fold 1/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Training fold 2/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Training fold 3/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Training fold 4/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Training fold 5/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

Results for 1. Chi-Square:
Average accura